# Reset_server

<div class="alert alert-block alert-info">
<i>Hello there.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> Here is an example to <b>reseting the server!!!</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style = "none">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

<div class="alert alert-block alert-success">
    <h1> 1.</h1>
    Declared a global variable i.e; DISABLE_RESOURCE_DIR.<br>
Below we are importing all the necessary modules from the ilorest library!.<br>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li>json </li>
    <li> redfish </li>
    <li> RedfishClient</li>
    <li> ServerDownOrUnreachableError </li>
    <li>get_resource_directory </li>
    <li> get_gen</li>
    
    
</div>

In [ ]:
import sys
import json
import argparse
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError
global DISABLE_RESOURCE_DIR
from ilorest_util import get_resource_directory
from ilorest_util import get_gen


<div class="alert alert-block alert-success">
    <p>This is our resetting_server function overview, in the end this function the server get restarted.</p>
    <p>Let me give you a walk through of it!</p>
    1.If either of no resource instance or DISABLE_RESOURCE_DIR flag is set then it retrieve the URI for systems from the root object, get the response for the systems URI using the Redfish API object, retrieve the URI for the members from the systems response object, and then get the response for the members URI using the Redfish API object.
    <br>
    <br>
    2.Else i.e;iterates through the resource instances, checking if the <b>@odata.type</b> value contains ComputerSystem. If found, it assigns the <b>@odata.id</b> value to <b>managers_members_uri </b>and retrieves the response using the Redfish API object.
    <br>
    <br>
    3.This code gets the URI for the computer system reset action, builds a dictionary with an action value and reset type, iterates through reset types, then posts the body to the URI for each reset type using the Redfish API object if managers_members_response exists.
    <br>
    <br>
    4.Next is to check the status of the response received from the previous <b>POST</b> request sent using the Redfish client object <b>_redfishobj</b>.
    <br>
        1.If the status is 400, it attempts to print the extended error message associated with the response.
    <br>
        2.If the response status is not 400 and not 200, an error message is printed to the standard error stream.
    <br>
        3.If the response status is 200, it indicates that the reset operation was successful. The code then prints           a success message and prints the JSON response received from the server in a formatted way.
    <br>
    <br>
    5. Again function call<b> reset_server_gen9(_redfishobj)</b> uses the Redfish API object to retrieve the response for the managers URI, the response object to retrieve the URI for the computer system reset action, etc. The action value and reset type are created in a dictionary,and the reset type is changed to <b>ForceRestart</b> after printing the URI.
     <br>
     Again the <b> POST</b> request sent by using RedFish client object is again checked.
</div>

In [ ]:
def reset_server(_redfishobj):
    
    managers_members_response = None
    #Assigns resource directory to variable using Redfish API object
    resource_instances = get_resource_directory(_redfishobj)
    #checking if no resource instance or DISABLE_RESOURCE_DIR flag is set 
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Systems']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if "ComputerSystem." in instance['@odata.type']:
                managers_members_uri = instance['@odata.id']
                managers_members_response = _redfishobj.get(managers_members_uri)
                
    if managers_members_response:
        path = managers_members_response.obj["Actions"]["#ComputerSystem.Reset"]["target"]
        body = dict()
        resettype = ['ForceRestart','GracefulRestart']
        body["Action"] = "ComputerSystem.Reset"
        for reset in resettype:
            if reset.lower() == "forcerestart":
                body['ResetType'] = "ForceRestart"
                resp = _redfishobj.post(path, body)
            elif reset.lower() == "gracefulrestart":
                body['ResetType'] = "GracefulRestart"
                resp = _redfishobj.post(path, body)

    #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
    #error message to see what went wrong
    if resp.status == 400:
        try:
            print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, sort_keys=True))
        except Exception as excp:
            sys.stderr.write("A response error occurred, unable to access iLO Extended Message "\
                             "Info...")
    elif resp.status != 200:
        sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
    else:
        print("Success!\n")
        print(json.dumps(resp.dict, indent=4, sort_keys=True))
        
#creates a dictionary containing an action value and reset type, then posts the dictionary to the 
#computer system reset action URI to reset a server using the Redfish API.
def reset_server_gen9(_redfishobj):
    
    managers_uri = "/redfish/v1/Systems/1/"
    managers_response = _redfishobj.get(managers_uri)
    system_path = managers_response.obj["Actions"]["#ComputerSystem.Reset"]["target"]
    print(system_path)
    body = dict()
    body["Action"] = "Reset"
    body["ResetType"] = "ForceRestart"

    resp = _redfishobj.post(system_path, body)
    #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
    #error message to see what went wrong
    if resp.status == 400:
        try:
            print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, sort_keys=True))
        except Exception as excp:
            sys.stderr.write("A response error occurred, unable to access iLO Extended Message "\
                             "Info...")
    elif resp.status != 200:
        sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
    else:
        print("Success!\n")
        print(json.dumps(resp.dict, indent=4, sort_keys=True))

<div class="alert alert-block alert-success">
<b> <h1> 3.</h1>The Main function: </b> <br>
    <ul>
        <li>After initilize the parser and adds argument such as  the iLO IP address, username, and password as three command-line options to the parser. Each argument has a corresponding default value and help text.</li>
        <li>In try section of code ;utilising the supplied iLO IP address, login, and password, to create a Redfish client object. If the login is successful, the <b>get_gen</b> function is used to check the server generation, and based on the generation number, the proper reset method is selected. The reset_server function is called if the server is a <b>Gen5 server</b>; otherwise, the <b>reset_server_gen9</b> function is called. An error message is printed to the console and the script terminates if the server cannot be reached or does not support Redfish.</li>
        <li> REDFISHOBJ.logout() will take care of logging you out of the ilo.</li>
    </ul>
</div>

In [1]:
if __name__ == "__main__":
    # Initialize parser 
    parser = argparse.ArgumentParser(description = "Script to upload and flash NVMe FW")    

    parser.add_argument(
        '-i',
        '--ilo',
        dest='ilo_ip',
        action="store",
        help="iLO IP of the server",
        default=None)
    parser.add_argument(
        '-u',
        '--user',
        dest='ilo_user',
        action="store",
        help="iLO username to login",
        default=None)
    parser.add_argument(
        '-p',
        '--password',
        dest='ilo_pass',
        action="store",
        help="iLO password to log in.",
        default=None)
    #stroing namespace object in options
    options = parser.parse_args()
   #getting namespaceobject ilo_ip address
    system_url = "https://" + options.ilo_ip
    #printing the  url obtain by ip
    print (system_url)

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = True

    try:
        # Create a Redfish client object
        redfish_obj = RedfishClient(base_url=system_url, username=options.ilo_user, password=options.ilo_pass)
        # Login with the Redfish client
        redfish_obj.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()
    
    (ilogen,_) = get_gen(redfish_obj)
    print ("Generation is ", ilogen)
    if int(ilogen) == 5:
        reset_server(redfish_obj)
    else:
        reset_server_gen9(redfish_obj)
    redfish_obj.logout()

NameError: name 'argparse' is not defined

<div class="alert alert-block alert-success">
    In case you need help 🔗 links to relevant resources :<br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
</div>